<a href="https://colab.research.google.com/github/joseluisspindola/RegresionLogistica/blob/main/Actividad_3_Regresi%C3%B3n_Log%C3%ADstica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8654]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8655]:
pd.options.display.max_columns = None

In [8656]:
data = pd.read_csv('Datos_limpios.csv', encoding='latin-1')
data.head(5)

,Unnamed: 0,id,host_id,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,price,minimum_nights,maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,last_scraped,source,name,host_name,host_since,host_location,host_response_time,host_is_superhost,host_neighbourhood,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,property_type,room_type,bathrooms,bathrooms_text,bedrooms,beds,amenities,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,has_availability,calendar_last_scraped,estimated_revenue_l365d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,0,35797.0,153786,94.2,88.9,1.0,1.0,19.38283,-99.27178,2,3799.0,1,7,1.0,7.0,29,59,89,364,0,0,0,188,0,0,1,1,0,0,26/06/2025,city scrape,Villa Dante,Dici,28/06/2010,"Mexico City, Mexico",Sin tiempo de respuesta promedio,0,Sin informaciÃ³n del vecindario,"['email', 'phone', 'work_email']",t,t,"Mexico City, D.f., Mexico",Cuajimalpa de Morelos,Entire villa,Entire home/apt,1.0,1 bath,1.0,1.0,"[""Kitchen"", ""Resort access"", ""Hot water"", ""Cou...",1.0,1.0,7.0,7.0,t,26/06/2025,0.0,No hay fechas,No hay fechas,0.00,0.00,0.00,0.00,0.00,0.00,0.00,f,50.00
1,1,44616.0,196253,100.0,91.0,13.0,13.0,19.41162,-99.17794,14,18000.0,1,180,1.0,180.0,29,59,89,360,65,1,0,179,0,6,9,4,2,0,01/07/2025,city scrape,Condesa Haus,Fernando,09/08/2010,"Mexico City, Mexico",within an hour,0,Condesa,"['email', 'phone', 'work_email']",t,t,Sin informacion,CuauhtÃÂ©moc,Entire home,Entire home/apt,5.5,5.5 baths,5.0,8.0,"[""Free street parking"", ""Free parking on premi...",1.0,1.0,180.0,180.0,t,01/07/2025,108000.0,09/11/2011,01/01/2025,4.59,4.56,4.70,4.87,4.78,4.98,4.47,f,0.39
2,2,56074.0,265650,100.0,100.0,1.0,5.0,19.43977,-99.15605,2,585.0,15,250,15.0,250.0,3,33,63,338,84,1,0,157,0,30,1,1,0,0,01/07/2025,city scrape,Great space in historical San Rafael,Maris,19/10/2010,"Mexico City, Mexico",within a few hours,0,San Rafael,"['email', 'phone']",t,t,"Mexico City, DF, Mexico",CuauhtÃÂ©moc,Entire condo,Entire home/apt,1.0,1 bath,1.0,1.0,"[""Dining table"", ""Hot water"", ""Hangers"", ""Esse...",15.0,15.0,250.0,250.0,t,01/07/2025,17550.0,02/04/2011,27/02/2025,4.87,4.95,4.88,4.98,4.94,4.76,4.79,f,0.48
3,3,67703.0,334451,100.0,47.0,3.0,4.0,19.41152,-99.16857,4,1696.0,2,30,2.0,30.0,3,4,32,267,50,1,0,97,4,6,2,2,0,0,01/07/2025,city scrape,"2 bedroom apt. deco bldg, Condesa",Nicholas,04/01/2011,"Mexico City, Mexico",within a few hours,0,HipÃÂ³dromo,"['email', 'phone']",t,t,Sin informacion,CuauhtÃÂ©moc,Entire rental unit,Entire home/apt,1.0,1 bath,2.0,2.0,"[""Hot water"", ""TV with standard cable"", ""Hange...",2.0,2.0,30.0,30.0,t,01/07/2025,10176.0,17/11/2011,30/10/2024,4.90,4.82,4.76,4.94,4.92,4.98,4.92,f,0.30
4,4,70644.0,212109,100.0,85.0,3.0,4.0,19.35448,-99.16217,2,1004.0,3,180,3.4,180.0,10,25,25,211,132,8,0,98,9,48,3,2,1,0,01/07/2025,city scrape,Beautiful light Studio Coyoacan- full equipped !,Trisha,24/08/2010,"Mexico City, Mexico",within a few hours,1,CoyoacÃÂ¡n,"['email', 'phone', 'work_email']",t,t,"Mexico City, Federal District, Mexico",CoyoacÃÂ¡n,Entire rental unit,Entire home/apt,1.0,1 bath,1.0,1.0,"[""Varies conditioner"", ""Dining table"", ""Free s...",3.0,4.0,180.0,180.0,t,01/07/2025,48192.0,14/02/2012,28/12/2024,4.92,4.91,4.96,4.96,4.98,4.96,4.92,f,0.81


In [8657]:
pd.options.display.max_rows = None
data.dtypes

,0
Unnamed: 0,int64
id,float64
host_id,int64
host_response_rate,float64
host_acceptance_rate,float64
host_listings_count,float64
host_total_listings_count,float64
latitude,float64
longitude,float64
accommodates,int64


In [8658]:
data['host_identity_verified']=data['host_identity_verified'].str.replace('f', '0')
data['host_identity_verified']=data['host_identity_verified'].str.replace('t', '1')
data['host_identity_verified']=data['host_identity_verified'].str.replace('Sin in0ormacion', '0')
data['host_identity_verified']= data['host_identity_verified'].astype(float)

In [8659]:
data['has_availability']=data['has_availability'].str.replace('f', '0')
data['has_availability']=data['has_availability'].str.replace('t', '1')
data['has_availability']=data['has_availability'].str.replace('Sin in0ormacion', '0')
data['has_availability']= data['has_availability'].astype(float)

In [8660]:
data['host_has_profile_pic']=data['host_has_profile_pic'].str.replace('f', '0')
data['host_has_profile_pic']=data['host_has_profile_pic'].str.replace('t', '1')
data['host_has_profile_pic']=data['host_has_profile_pic'].str.replace('Sin in0ormacion', '0')
data['host_has_profile_pic']= data['host_has_profile_pic'].astype(float)

In [8661]:
cuantitativas= data.select_dtypes(include=['float64','int64','float','int'])
cualitativas=data.select_dtypes(include=['object','datetime','category'])

In [8662]:
Corr_Factors_cuantitativas=cuantitativas.corr()
Corr_Factors_cuantitativas

,Unnamed: 0,id,host_id,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,price,minimum_nights,maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,host_is_superhost,host_has_profile_pic,host_identity_verified,bathrooms,bedrooms,beds,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,has_availability,estimated_revenue_l365d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
Unnamed: 0,1.000000,0.960946,0.443458,0.026323,0.066636,0.081468,0.078770,0.103985,0.065279,0.035397,0.019110,-0.058569,-0.257044,-0.065696,-0.012444,0.158456,0.178761,0.185601,0.170296,-0.395095,-0.023951,0.094093,0.182614,-0.135582,-0.043531,0.123222,0.102325,0.077490,0.078010,-0.023216,-0.071312,0.096311,-0.162445,-0.102972,-0.166125,-0.058092,-0.055404,-0.012444,-0.012444,0.106439,-0.110691,-0.178662,-0.180414,-0.176920,-0.179890,-0.181026,-0.173314,-0.180572,0.187180
id,0.960946,1.000000,0.418664,0.022889,0.061157,0.084718,0.082628,0.103610,0.056262,0.038561,0.020412,-0.056058,-0.265750,-0.065039,-0.023037,0.148839,0.169333,0.176454,0.165700,-0.377797,-0.009934,0.093488,0.174761,-0.115958,-0.023304,0.130407,0.111605,0.070742,0.070257,-0.007449,-0.071178,0.094409,-0.157407,-0.100980,-0.161954,-0.058108,-0.054161,-0.023037,-0.023037,0.103647,-0.094215,-0.153619,-0.155624,-0.151825,-0.154953,-0.156130,-0.148551,-0.155804,0.162080
host_id,0.443458,0.418664,1.000000,-0.029505,0.032732,-0.065252,-0.097102,0.042150,0.098879,-0.000509,0.032491,-0.035367,-0.077949,-0.045648,0.012181,0.130143,0.132863,0.130378,0.117423,-0.175695,0.016665,0.069270,0.135564,-0.035796,-0.005553,0.022383,0.010421,0.057108,-0.026048,-0.073516,-0.122170,0.015368,-0.049739,-0.036966,-0.049925,-0.039689,-0.046741,0.012180,0.012181,-0.003253,-0.042539,-0.108884,-0.107989,-0.104527,-0.105900,-0.110844,-0.105307,-0.108997,0.106189
host_response_rate,0.026323,0.022889,-0.029505,1.000000,0.542207,0.054524,0.052079,0.055994,0.029537,0.059571,0.009216,-0.037400,-0.008903,-0.042921,-0.000002,-0.101151,-0.086387,-0.077918,-0.022426,0.132224,0.167622,0.126975,-0.051934,0.144603,0.217527,0.062099,0.060673,0.004388,0.016941,0.196164,0.035557,0.001413,-0.030136,-0.004294,-0.027400,-0.034608,-0.026275,-0.000002,-0.000002,0.065300,0.092619,0.187644,0.187871,0.191195,0.185575,0.189336,0.183832,0.185858,-0.161864
host_acceptance_rate,0.066636,0.061157,0.032732,0.542207,1.000000,0.092504,0.091549,0.077766,0.002771,0.044000,-0.001983,-0.035405,0.015580,-0.041167,0.000036,-0.094800,-0.070382,-0.062552,0.006746,0.197561,0.249730,0.177229,-0.026151,0.223696,0.317888,0.110396,0.112614,-0.001152,-0.001408,0.251891,0.043141,0.006220,-0.047059,-0.010000,-0.046056,-0.030944,-0.021966,0.000036,0.000036,0.074524,0.123252,0.226015,0.228009,0.230961,0.226887,0.225402,0.227483,0.225498,-0.199670
host_listings_count,0.081468,0.084718,-0.065252,0.054524,0.092504,1.000000,0.889168,0.088615,-0.057178,0.052586,0.005417,0.000599,0.058124,-0.025660,-0.004745,0.009995,0.029312,0.037126,0.108683,0.042676,0.087327,0.102653,0.066811,0.075922,0.118087,0.706346,0.701433,0.053490,0.052594,-0.065924,0.060297,0.070444,-0.064820,-0.034632,-0.065865,-0.022124,-0.018428,-0.004745,-0.004745,0.049291,0.052246,0.040134,0.045542,0.047981,0.046851,0.039022,0.057155,0.040513,-0.048692
host_total_listings_count,0.078770,0.082628,-0.097102,0.052079,0.091549,0.889168,1.000000,0.081398,-0.053584,0.067627,0.007800,-0.004623,0.069521,-0.026159,-0.004660,-0.0

Caso 1

In [8663]:
Vars_Indep1= data[['host_response_rate','host_acceptance_rate','review_scores_rating']]
Var_Dep1= data['host_is_superhost']

In [8664]:
X1= Vars_Indep1
y1= Var_Dep1

In [8665]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state =None)

In [8666]:
escalar1 = StandardScaler()

In [8667]:
X1_train = escalar1.fit_transform(X1_train)
X1_test = escalar1.transform(X1_test)

In [8668]:
from sklearn.linear_model import LogisticRegression
algoritmo1 = LogisticRegression()

In [8669]:
algoritmo1.fit(X1_train, y1_train)

LogisticRegression()

In [8670]:
y1_pred = algoritmo1.predict(X1_test)
y1_pred

array([1, 0, 1, ..., 1, 0, 0])

In [8671]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y1_test, y1_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[4183  886]
 [1258 1594]]


In [8672]:
from sklearn.metrics import precision_score
precision1 = precision_score(y1_test, y1_pred, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision1)

Precisión del modelo:
0.768792501378423


In [8673]:
precision2 = precision_score(y1_test, y1_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision2)

Precisión del modelo:
0.6427419354838709


In [8674]:
from sklearn.metrics import accuracy_score

exactitud1 = accuracy_score(y1_test, y1_pred)
print('Exactitud del modelo:')
print(exactitud1)

Exactitud del modelo:
0.7293271051634894


In [8675]:
from sklearn.metrics import recall_score

sensibilidad1 = recall_score(y1_test, y1_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad1)

Sensibilidad del modelo:
0.8252120733872559


In [8676]:
sensibilidad2 = recall_score(y1_test, y1_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad2)

Sensibilidad del modelo:
0.55890603085554


In [8677]:
from sklearn.metrics import f1_score

puntajef1_1 = f1_score(y1_test, y1_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1_1)

Puntaje F1 del modelo:
0.5978994748687172


In [8678]:
puntajef1_2 = f1_score(y1_test, y1_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1_2)

Puntaje F1 del modelo:
0.5978994748687172


Caso 2

In [8679]:
Max=data['price'].max()
Min=data['price'].min()
Limites= [Min, Max]
Limites

[66.0, 900000.0]

In [8680]:
intervalos2=np.linspace(65.9, 900000.1, 3)
intervalos2

array([6.590000e+01, 4.500330e+05, 9.000001e+05])

In [8681]:
data['price'].mean()

np.float64(1989.29340555282)

In [8682]:
limites2=[65.8, 1500, 900000.1]
limites2

[65.8, 1500, 900000.1]

In [8683]:
categorias2= ["Económico", "De lujo"]

In [8684]:
data['price'] = pd.to_numeric(data['price'], errors='coerce')

data['price']=pd.cut(x= data['price'], bins=limites2, labels= categorias2 )

In [8685]:
Vars_Indep2= data[['estimated_revenue_l365d','review_scores_value','reviews_per_month']]
Var_Dep2= data['price']

In [8686]:
X2= Vars_Indep2
y2= Var_Dep2

In [8687]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state =None)

In [8688]:
escalar2 = StandardScaler()

In [8689]:
X2_train = escalar2.fit_transform(X2_train)
X2_test = escalar2.transform(X2_test)

In [8690]:
from sklearn.linear_model import LogisticRegression
algoritmo2 = LogisticRegression()

In [8691]:
algoritmo2.fit(X2_train, y2_train)

LogisticRegression()

In [8692]:
y2_pred = algoritmo2.predict(X2_test)
y2_pred

array(['De lujo', 'Económico', 'Económico', ..., 'Económico', 'De lujo',
       'Económico'], dtype=object)

In [8693]:
matriz2 = confusion_matrix(y2_test, y2_pred)
print('Matriz de Confusión:')
print(matriz2)

Matriz de Confusión:
[[1725 1262]
 [ 353 4581]]


In [8694]:
precision3 = precision_score(y2_test, y2_pred, average="binary", pos_label='Económico')
print('Precisión del modelo:')
print(precision3)

Precisión del modelo:
0.7840150607564608


In [8695]:
precision4 = precision_score(y2_test, y2_pred, average="binary", pos_label='De lujo')
print('Precisión del modelo:')
print(precision4)

Precisión del modelo:
0.8301251203079885


In [8696]:

exactitud2 = accuracy_score(y2_test, y2_pred)
print('Exactitud del modelo:')
print(exactitud2)

Exactitud del modelo:
0.7961116020704456


In [8697]:
sensibilidad3 = recall_score(y2_test, y2_pred, average="binary", pos_label='Económico')
print('Sensibilidad del modelo:')
print(sensibilidad3)

Sensibilidad del modelo:
0.9284556141062018


In [8698]:

sensibilidad4 = recall_score(y2_test, y2_pred, average="binary", pos_label='De lujo')
print('Sensibilidad del modelo:')
print(sensibilidad4)

Sensibilidad del modelo:
0.5775025108804821


In [8699]:
puntajef1_3 = f1_score(y2_test, y2_pred, average="binary", pos_label='Económico')
print('Puntaje F1 del modelo:')
print(puntajef1_3)

Puntaje F1 del modelo:
0.8501438248120998


In [8700]:
puntajef1_4 = f1_score(y2_test, y2_pred, average="binary", pos_label='De lujo')
print('Puntaje F1 del modelo:')
print(puntajef1_4)

Puntaje F1 del modelo:
0.6811451135241856


Caso 3

In [8701]:
Max=data['availability_30'].max()
Min=data['availability_30'].min()
Limites= [Min, Max]
Limites

[0, 30]

In [8702]:
intervalos3=np.linspace(-0.1, 30.1, 3)
intervalos3

array([-0.1, 15. , 30.1])

In [8703]:
categorias3= ["Poca disponibilidad", "Disponible"]

In [8704]:
data['availability_30'] = pd.to_numeric(data['availability_30'], errors='coerce')

data['availability_30']=pd.cut(x= data['availability_30'], bins=intervalos3, labels= categorias3 )

In [8705]:
Vars_Indep3= data[['availability_60','estimated_occupancy_l365d','bedrooms']]
Var_Dep3= data['availability_30']

In [8706]:
X3= Vars_Indep3
y3= Var_Dep3

In [8707]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.3, random_state =None)

In [8708]:
escalar3 = StandardScaler()

In [8709]:
X3_train = escalar3.fit_transform(X3_train)
X3_test = escalar3.transform(X3_test)

In [8710]:
from sklearn.linear_model import LogisticRegression
algoritmo3 = LogisticRegression()

In [8711]:
algoritmo3.fit(X3_train, y3_train)

LogisticRegression()

In [8712]:
y3_pred = algoritmo3.predict(X3_test)
y3_pred

array(['Disponible', 'Poca disponibilidad', 'Disponible', ...,
       'Poca disponibilidad', 'Disponible', 'Poca disponibilidad'],
      dtype=object)

In [8713]:
matriz3 = confusion_matrix(y3_test, y3_pred)
print('Matriz de Confusión:')
print(matriz3)

Matriz de Confusión:
[[3970  267]
 [ 399 3285]]


In [8714]:
precision5 = precision_score(y3_test, y3_pred, average="binary", pos_label='Poca disponibilidad')
print('Precisión del modelo:')
print(precision5)

Precisión del modelo:
0.924831081081081


In [8715]:
precision6 = precision_score(y3_test, y3_pred, average="binary", pos_label='Disponible')
print('Precisión del modelo:')
print(precision6)

Precisión del modelo:
0.9086747539482719


In [8716]:
exactitud3 = accuracy_score(y3_test, y3_pred)
print('Exactitud del modelo:')
print(exactitud3)

Exactitud del modelo:
0.9159197071076884


In [8717]:
sensibilidad5 = recall_score(y3_test, y3_pred, average="binary", pos_label='Poca disponibilidad')
print('Sensibilidad del modelo:')
print(sensibilidad5)

Sensibilidad del modelo:
0.8916938110749185


In [8718]:
sensibilidad6 = recall_score(y3_test, y3_pred, average="binary", pos_label='Disponible')
print('Sensibilidad del modelo:')
print(sensibilidad6)

Sensibilidad del modelo:
0.9369837148926127


In [8719]:
puntajef1_5 = f1_score(y3_test, y3_pred, average="binary", pos_label='Poca disponibilidad')
print('Puntaje F1 del modelo:')
print(puntajef1_5)

Puntaje F1 del modelo:
0.9079601990049752


In [8720]:
puntajef1_6 = f1_score(y3_test, y3_pred, average="binary", pos_label='Disponible')
print('Puntaje F1 del modelo:')
print(puntajef1_6)

Puntaje F1 del modelo:
0.9226121310713455


Caso 4

In [8721]:
Max=data['accommodates'].max()
Min=data['accommodates'].min()
Limites= [Min, Max]
Limites

[1, 16]

In [8722]:
intervalos4=np.linspace(0.9, 16.1, 3)
intervalos4

array([ 0.9,  8.5, 16.1])

In [8723]:
data['accommodates'].mean()

np.float64(3.333434339608348)

In [8724]:
limites4=[0.9, 4, 16.1]
limites4

[0.9, 4, 16.1]

In [8725]:
categorias4= ["Chico", "Grande"]

In [8726]:
data['accommodates'] = pd.to_numeric(data['accommodates'], errors='coerce')

data['accommodates']=pd.cut(x= data['accommodates'], bins=limites4, labels= categorias4 )

In [8727]:
Vars_Indep4= data[['bathrooms','estimated_revenue_l365d','calculated_host_listings_count_private_rooms']]
Var_Dep4= data['accommodates']

In [8728]:
X4= Vars_Indep4
y4= Var_Dep4

In [8729]:
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.3, random_state =None)

In [8730]:
escalar4 = StandardScaler()

In [8731]:
X4_train = escalar4.fit_transform(X4_train)
X4_test = escalar4.transform(X4_test)

In [8732]:
from sklearn.linear_model import LogisticRegression
algoritmo4 = LogisticRegression()

In [8733]:
algoritmo4.fit(X4_train, y4_train)

LogisticRegression()

In [8734]:
y4_pred = algoritmo4.predict(X4_test)
y4_pred

array(['Chico', 'Chico', 'Chico', ..., 'Chico', 'Chico', 'Chico'],
      dtype=object)

In [8735]:
matriz4 = confusion_matrix(y4_test, y4_pred)
print('Matriz de Confusión:')
print(matriz4)

Matriz de Confusión:
[[6401   61]
 [1333  126]]


In [8736]:
precision7 = precision_score(y4_test, y4_pred, average="binary", pos_label='Chico')
print('Precisión del modelo:')
print(precision7)

Precisión del modelo:
0.8276441686061546


In [8737]:
precision8 = precision_score(y4_test, y4_pred, average="binary", pos_label='Grande')
print('Precisión del modelo:')
print(precision8)

Precisión del modelo:
0.6737967914438503


In [8738]:
exactitud4 = accuracy_score(y4_test, y4_pred)
print('Exactitud del modelo:')
print(exactitud4)

Exactitud del modelo:
0.8240121196818584


In [8739]:
sensibilidad7 = recall_score(y4_test, y4_pred, average="binary", pos_label='Chico')
print('Sensibilidad del modelo:')
print(sensibilidad7)

Sensibilidad del modelo:
0.9905601980810894


In [8740]:
sensibilidad8 = recall_score(y4_test, y4_pred, average="binary", pos_label='Grande')
print('Sensibilidad del modelo:')
print(sensibilidad8)

Sensibilidad del modelo:
0.08636052090472926


In [8741]:
puntajef1_7 = f1_score(y4_test, y4_pred, average="binary", pos_label='Chico')
print('Puntaje F1 del modelo:')
print(puntajef1_7)

Puntaje F1 del modelo:
0.901803324880248


In [8742]:
puntajef1_8 = f1_score(y4_test, y4_pred, average="binary", pos_label='Grande')
print('Puntaje F1 del modelo:')
print(puntajef1_8)

Puntaje F1 del modelo:
0.15309842041312272


Caso 5

In [8743]:
Vars_Indep5= data[['bathrooms','bedrooms','beds']]
Var_Dep5= data['host_identity_verified']

In [8744]:
X5= Vars_Indep5
y5= Var_Dep5

In [8745]:
X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5, test_size=0.3, random_state =None)

In [8746]:
escalar5 = StandardScaler()

In [8747]:
X5_train = escalar5.fit_transform(X5_train)
X5_test = escalar5.transform(X5_test)

In [8748]:
from sklearn.linear_model import LogisticRegression
algoritmo5 = LogisticRegression()

In [8749]:
algoritmo5.fit(X5_train, y5_train)

LogisticRegression()

In [8750]:
y5_pred = algoritmo5.predict(X5_test)
y5_pred

array([1., 1., 1., ..., 1., 1., 1.])

In [8751]:
matriz5 = confusion_matrix(y5_test, y5_pred)
print('Matriz de Confusión:')
print(matriz5)

Matriz de Confusión:
[[   0  634]
 [   0 7287]]


In [8752]:
precision9 = precision_score(y5_test, y5_pred, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision9)

Precisión del modelo:
0.0


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8753]:
precision10 = precision_score(y5_test, y5_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision10)

Precisión del modelo:
0.9199596010604721


In [8754]:
exactitud5 = accuracy_score(y5_test, y5_pred)
print('Exactitud del modelo:')
print(exactitud5)

Exactitud del modelo:
0.9199596010604721


In [8755]:
sensibilidad9 = recall_score(y5_test, y5_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad9)

Sensibilidad del modelo:
0.0


In [8756]:
sensibilidad10 = recall_score(y5_test, y5_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad10)

Sensibilidad del modelo:
1.0


In [8757]:
puntajef1_9 = f1_score(y5_test, y5_pred, average="binary", pos_label=0)
print('Puntaje F1 del modelo:')
print(puntajef1_9)

Puntaje F1 del modelo:
0.0


In [8758]:
puntajef1_10 = f1_score(y5_test, y5_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1_10)

Puntaje F1 del modelo:
0.9583114150447133


In [8858]:
pip install imblearn

In [8859]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X5_resampled, y5_resampled = smote.fit_resample(X5_train, y5_train)

In [8860]:
algoritmo5_Over_sampling = LogisticRegression()
algoritmo5_Over_sampling.fit(X5_resampled, y5_resampled)

LogisticRegression()

In [8861]:
y5_pred_over_sampling = algoritmo5_Over_sampling.predict(X5_test)
y5_pred_over_sampling

array([1., 1., 1., ..., 1., 1., 1.])

In [8862]:
from sklearn.metrics import confusion_matrix
matriz5 = confusion_matrix(y5_test, y5_pred_over_sampling)
print('Matriz de Confusión:')
print(matriz5)

Matriz de Confusión:
[[ 242  392]
 [ 777 6510]]


In [8863]:
precision9o = precision_score(y5_test, y5_pred_over_sampling, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision9o)

Precisión del modelo:
0.23748773307163887


In [8864]:
precision10o = precision_score(y5_test, y5_pred_over_sampling, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision10o)

Precisión del modelo:
0.9432048681541582


In [8865]:
exactitud5o = accuracy_score(y5_test, y5_pred_over_sampling)
print('Exactitud del modelo:')
print(exactitud5o)

Exactitud del modelo:
0.852417624037369


In [8867]:
sensibilidad9o = recall_score(y5_test, y5_pred_over_sampling, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad9o)

Sensibilidad del modelo:
0.38170347003154576


In [8868]:
sensibilidad10o = recall_score(y5_test, y5_pred_over_sampling, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad10o)

Sensibilidad del modelo:
0.8933717579250721


Caso 6

In [8759]:
Max=data['calculated_host_listings_count'].max()
Min=data['calculated_host_listings_count'].min()
Limites= [Min, Max]
Limites

[1, 235]

In [8760]:
intervalos6=np.linspace(0.9, 235.1, 3)
intervalos6

array([  0.9, 118. , 235.1])

In [8761]:
limites6=[0.9, 3, 235.1]
limites6

[0.9, 3, 235.1]

In [8762]:
categorias6= ["Pocas propiedades", "Muchas propiedades"]

In [8763]:
data['calculated_host_listings_count'] = pd.to_numeric(data['calculated_host_listings_count'], errors='coerce')

data['calculated_host_listings_count']=pd.cut(x= data['calculated_host_listings_count'], bins=limites6, labels= categorias6)

In [8764]:
Vars_Indep6= data[['host_listings_count','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms']]
Var_Dep6= data['calculated_host_listings_count']

In [8765]:
X6= Vars_Indep6
y6= Var_Dep6

In [8766]:
X6_train, X6_test, y6_train, y6_test = train_test_split(X6, y6, test_size=0.3, random_state =None)

In [8767]:
escalar6 = StandardScaler()

In [8768]:
X6train = escalar6.fit_transform(X6_train)
X6_test = escalar6.transform(X6_test)

In [8769]:
from sklearn.linear_model import LogisticRegression
algoritmo6 = LogisticRegression()

In [8770]:
algoritmo6.fit(X6_train, y6_train)

LogisticRegression()

In [8771]:
y6_pred = algoritmo6.predict(X6_test)
y6_pred

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array(['Pocas propiedades', 'Pocas propiedades', 'Pocas propiedades', ...,
       'Pocas propiedades', 'Pocas propiedades', 'Pocas propiedades'],
      dtype=object)

In [8772]:
matriz6 = confusion_matrix(y6_test, y6_pred)
print('Matriz de Confusión:')
print(matriz6)

Matriz de Confusión:
[[ 359 3477]
 [   0 4085]]


In [8773]:
precision11 = precision_score(y6_test, y6_pred, average="binary", pos_label="Pocas propiedades")
print('Precisión del modelo:')
print(precision11)

Precisión del modelo:
0.5402010050251256


In [8774]:
precision12 = precision_score(y6_test, y6_pred, average="binary", pos_label="Muchas propiedades")
print('Precisión del modelo:')
print(precision12)

Precisión del modelo:
1.0


In [8775]:
exactitud6 = accuracy_score(y6_test, y6_pred)
print('Exactitud del modelo:')
print(exactitud6)

Exactitud del modelo:
0.5610402726928418


In [8776]:
sensibilidad11 = recall_score(y6_test, y6_pred, average="binary", pos_label="Pocas propiedades")
print('Sensibilidad del modelo:')
print(sensibilidad11)

Sensibilidad del modelo:
1.0


In [8777]:
sensibilidad12 = recall_score(y6_test, y6_pred, average="binary", pos_label="Muchas propiedades")
print('Sensibilidad del modelo:')
print(sensibilidad12)

Sensibilidad del modelo:
0.09358706986444212


In [8778]:
puntajef1_11 = f1_score(y6_test, y6_pred, average="binary", pos_label="Pocas propiedades")
print('Puntaje F1 del modelo:')
print(puntajef1_11)

Puntaje F1 del modelo:
0.7014681892332789


In [8779]:
puntajef1_12 = f1_score(y6_test, y6_pred, average="binary", pos_label="Muchas propiedades")
print('Puntaje F1 del modelo:')
print(puntajef1_12)

Puntaje F1 del modelo:
0.17115613825983314


Caso 7

In [8780]:
Vars_Indep7= data[['host_identity_verified','host_id','review_scores_accuracy']]
Var_Dep7= data['host_has_profile_pic']

In [8781]:
X7= Vars_Indep7
y7= Var_Dep7

In [8782]:
X7_train, X7_test, y7_train, y7_test = train_test_split(X7, y7, test_size=0.3, random_state =None)

In [8783]:
escalar7 = StandardScaler()

In [8784]:
X7train = escalar7.fit_transform(X7_train)
X7_test = escalar7.transform(X7_test)

In [8785]:
algoritmo7 = LogisticRegression()

In [8786]:
algoritmo7.fit(X7_train, y7_train)

LogisticRegression()

In [8787]:
y7_pred = algoritmo7.predict(X7_test)
y7_pred

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([1., 1., 1., ..., 0., 1., 1.])

In [8788]:
matriz7 = confusion_matrix(y7_test, y7_pred)
print('Matriz de Confusión:')
print(matriz7)

Matriz de Confusión:
[[ 148  287]
 [4489 2997]]


In [8789]:
precision13 = precision_score(y7_test, y7_pred, average="binary", pos_label=0)
print('Precisión del modelo:')
print(precision13)

Precisión del modelo:
0.031917187836963554


In [8790]:
precision14 = precision_score(y7_test, y7_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision14)

Precisión del modelo:
0.9126065773447016


In [8791]:
exactitud7 = accuracy_score(y7_test, y7_pred)
print('Exactitud del modelo:')
print(exactitud7)

Exactitud del modelo:
0.3970458275470269


In [8792]:
sensibilidad13 = recall_score(y7_test, y7_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad13)

Sensibilidad del modelo:
0.34022988505747126


In [8793]:
sensibilidad14 = recall_score(y7_test, y7_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad14)

Sensibilidad del modelo:
0.40034731498797754


In [8794]:
puntajef1_13 = f1_score(y7_test, y7_pred, average="binary", pos_label=0)
print('Puntaje F1 del modelo:')
print(puntajef1_13)

Puntaje F1 del modelo:
0.0583596214511041


In [8795]:
puntajef1_14 = f1_score(y7_test, y7_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1_14)

Puntaje F1 del modelo:
0.5565459610027855


Caso 8

In [8796]:
Max=data['bedrooms'].max()
Min=data['bedrooms'].min()
Limites= [Min, Max]
Limites

[0.0, 50.0]

In [8797]:
intervalos8=np.linspace(-0.1, 50.1, 3)
intervalos8

array([-0.1, 25. , 50.1])

In [8798]:
limites8=[-0.1, 5, 50.1]
limites8

[-0.1, 5, 50.1]

In [8799]:
categorias8= ["Pocos cuartos", "Muchos cuartos"]

In [8800]:
data['bedrooms'] = pd.to_numeric(data['bedrooms'], errors='coerce')

data['bedrooms']=pd.cut(x= data['bedrooms'], bins=limites8, labels= categorias8)

In [8801]:
Vars_Indep8= data[['bathrooms','beds','has_availability']]
Var_Dep8= data['bedrooms']

In [8802]:
X8= Vars_Indep8
y8= Var_Dep8

In [8803]:
X8_train, X8_test, y8_train, y8_test = train_test_split(X8, y8, test_size=0.3, random_state =None)

In [8804]:
escalar8 = StandardScaler()

In [8805]:
X8train = escalar8.fit_transform(X8_train)
X8_test = escalar8.transform(X8_test)

In [8806]:
algoritmo8 = LogisticRegression()

In [8807]:
from sklearn.linear_model import LogisticRegression
algoritmo8.fit(X8_train, y8_train)

LogisticRegression()

In [8808]:
y8_pred = algoritmo8.predict(X8_test)
y8_pred

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array(['Pocos cuartos', 'Pocos cuartos', 'Pocos cuartos', ...,
       'Pocos cuartos', 'Pocos cuartos', 'Pocos cuartos'], dtype=object)

In [8809]:
matriz8 = confusion_matrix(y8_test, y8_pred)
print('Matriz de Confusión:')
print(matriz8)

Matriz de Confusión:
[[ 102  246]
 [ 196 7377]]


In [8810]:
precision15 = precision_score(y8_test, y8_pred, average="binary", pos_label="Pocos cuartos")
print('Precisión del modelo:')
print(precision15)

Precisión del modelo:
0.9677292404565132


In [8811]:
precision16 = precision_score(y8_test, y8_pred, average="binary", pos_label="Muchos cuartos")
print('Precisión del modelo:')
print(precision16)

Precisión del modelo:
0.3422818791946309


In [8812]:
exactitud8 = accuracy_score(y8_test, y8_pred)
print('Exactitud del modelo:')
print(exactitud8)

Exactitud del modelo:
0.9441989647771746


In [8813]:
sensibilidad15 = recall_score(y8_test, y8_pred, average="binary", pos_label="Pocos cuartos")
print('Sensibilidad del modelo:')
print(sensibilidad15)

Sensibilidad del modelo:
0.9741185791628153


In [8814]:
sensibilidad16 = recall_score(y8_test, y8_pred, average="binary", pos_label="Muchos cuartos")
print('Sensibilidad del modelo:')
print(sensibilidad16)

Sensibilidad del modelo:
0.29310344827586204


In [8815]:
puntajef1_15 = f1_score(y8_test, y8_pred, average="binary", pos_label="Pocos cuartos")
print('Puntaje F1 del modelo:')
print(puntajef1_15)

Puntaje F1 del modelo:
0.9709133982627007


In [8816]:
puntajef1_16 = f1_score(y8_test, y8_pred, average="binary", pos_label="Muchos cuartos")
print('Puntaje F1 del modelo:')
print(puntajef1_16)

Puntaje F1 del modelo:
0.3157894736842105


Caso 9

In [8817]:
Max=data['estimated_revenue_l365d'].max()
Min=data['estimated_revenue_l365d'].min()
Limites= [Min, Max]
Limites

[0.0, 10800000.0]

In [8818]:
intervalos9=np.linspace(-0.1, 10800000.0, 3)
intervalos9

array([-1.00000000e-01,  5.39999995e+06,  1.08000000e+07])

In [8819]:
limites9=[-0.1, 10000, 10800000.1]
limites9

[-0.1, 10000, 10800000.1]

In [8820]:
categorias9= ["Bajos ingresos", "Altos ingresos"]

In [8821]:
data['estimated_revenue_l365d'] = pd.to_numeric(data['estimated_revenue_l365d'], errors='coerce')

data['estimated_revenue_l365d']=pd.cut(x= data['estimated_revenue_l365d'], bins=limites9, labels= categorias9)

In [8822]:
Vars_Indep9= data[['review_scores_value','review_scores_location','has_availability']]
Var_Dep9= data['estimated_revenue_l365d']

In [8823]:
X9= Vars_Indep9
y9= Var_Dep9

In [8824]:
X9_train, X9_test, y9_train, y9_test = train_test_split(X9, y9, test_size=0.3, random_state =None)

In [8825]:
escalar9 = StandardScaler()

In [8826]:
X9train = escalar9.fit_transform(X9_train)
X9_test = escalar9.transform(X9_test)

In [8827]:
algoritmo9 = LogisticRegression()

In [8828]:
from sklearn.linear_model import LogisticRegression
algoritmo9.fit(X9_train, y9_train)

LogisticRegression()

In [8829]:
y9_pred = algoritmo9.predict(X9_test)
y9_pred

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array(['Altos ingresos', 'Altos ingresos', 'Altos ingresos', ...,
       'Altos ingresos', 'Altos ingresos', 'Altos ingresos'], dtype=object)

In [8830]:
matriz9 = confusion_matrix(y9_test, y9_pred)
print('Matriz de Confusión:')
print(matriz9)

Matriz de Confusión:
[[5840  156]
 [1203  722]]


In [8831]:
precision17 = precision_score(y9_test, y9_pred, average="binary", pos_label="Bajos ingresos")
print('Precisión del modelo:')
print(precision17)

Precisión del modelo:
0.8223234624145785


In [8832]:
precision18 = precision_score(y9_test, y9_pred, average="binary", pos_label="Altos ingresos")
print('Precisión del modelo:')
print(precision18)

Precisión del modelo:
0.8291921056368025


In [8833]:
exactitud9 = accuracy_score(y9_test, y9_pred)
print('Exactitud del modelo:')
print(exactitud9)

Exactitud del modelo:
0.8284307536927156


In [8834]:
sensibilidad17 = recall_score(y9_test, y9_pred, average="binary", pos_label="Bajos ingresos")
print('Sensibilidad del modelo:')
print(sensibilidad17)

Sensibilidad del modelo:
0.37506493506493505


In [8835]:
sensibilidad18 = recall_score(y9_test, y9_pred, average="binary", pos_label="Altos ingresos")
print('Sensibilidad del modelo:')
print(sensibilidad18)

Sensibilidad del modelo:
0.9739826551034023


In [8836]:
puntajef1_17 = f1_score(y9_test, y9_pred, average="binary", pos_label="Bajos ingresos")
print('Puntaje F1 del modelo:')
print(puntajef1_17)

Puntaje F1 del modelo:
0.5151623260792009


In [8837]:
puntajef1_18 = f1_score(y9_test, y9_pred, average="binary", pos_label="Altos ingresos")
print('Puntaje F1 del modelo:')
print(puntajef1_18)

Puntaje F1 del modelo:
0.8957742158140962


Caso 10

In [8838]:
Max=data['review_scores_rating'].max()
Min=data['review_scores_rating'].min()
Limites= [Min, Max]
Limites

[0.0, 5.0]

In [8839]:
intervalos10=np.linspace(-0.1, 5.1, 3)
intervalos10

array([-0.1,  2.5,  5.1])

In [8840]:
categorias10= ["Baja calificacion", "Buena calificacion"]

In [8841]:
data['review_scores_rating'] = pd.to_numeric(data['review_scores_rating'], errors='coerce')

data['review_scores_rating']=pd.cut(x= data['review_scores_rating'], bins=intervalos10, labels= categorias10)

In [8842]:
Vars_Indep10= data[['bathrooms','number_of_reviews','number_of_reviews_ltm']]
Var_Dep10= data['review_scores_rating']

In [8843]:
X10= Vars_Indep10
y10= Var_Dep10

In [8844]:
X10_train, X10_test, y10_train, y10_test = train_test_split(X10, y10, test_size=0.3, random_state =None)

In [8845]:
escalar10 = StandardScaler()

In [8846]:
X10train = escalar10.fit_transform(X10_train)
X10_test = escalar10.transform(X10_test)

In [8847]:
from sklearn.linear_model import LogisticRegression
algoritmo10 = LogisticRegression()

In [8848]:
algoritmo10.fit(X10_train, y10_train)

LogisticRegression()

In [8849]:
y10_pred = algoritmo10.predict(X10_test)
y10_pred

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array(['Buena calificacion', 'Buena calificacion', 'Baja calificacion',
       ..., 'Baja calificacion', 'Baja calificacion', 'Baja calificacion'],
      dtype=object)

In [8850]:
matriz10 = confusion_matrix(y10_test, y10_pred)
print('Matriz de Confusión:')
print(matriz10)

Matriz de Confusión:
[[1004    0]
 [5682 1235]]


In [8851]:
precision19 = precision_score(y10_test, y10_pred, average="binary", pos_label="Baja calificacion")
print('Precisión del modelo:')
print(precision19)

Precisión del modelo:
0.15016452288363746


In [8852]:
precision20 = precision_score(y10_test, y10_pred, average="binary", pos_label="Buena calificacion")
print('Precisión del modelo:')
print(precision20)

Precisión del modelo:
1.0


In [8853]:
exactitud10 = accuracy_score(y1_test, y1_pred)
print('Exactitud del modelo:')
print(exactitud10)

Exactitud del modelo:
0.7293271051634894


In [8854]:
sensibilidad19 = recall_score(y10_test, y10_pred, average="binary", pos_label="Baja calificacion")
print('Sensibilidad del modelo:')
print(sensibilidad19)

Sensibilidad del modelo:
1.0


In [8855]:
sensibilidad20 = recall_score(y10_test, y10_pred, average="binary", pos_label="Buena calificacion")
print('Sensibilidad del modelo:')
print(sensibilidad20)

Sensibilidad del modelo:
0.17854561225965013


In [8856]:
puntajef1_19 = f1_score(y10_test, y10_pred, average="binary", pos_label="Baja calificacion")
print('Puntaje F1 del modelo:')
print(puntajef1_19)

Puntaje F1 del modelo:
0.2611183355006502


In [8857]:
puntajef1_20 = f1_score(y10_test, y10_pred, average="binary", pos_label="Buena calificacion")
print('Puntaje F1 del modelo:')
print(puntajef1_20)

Puntaje F1 del modelo:
0.3029931305201178
